In [73]:
import pandas as pd
import numpy as np
import os

In [74]:
%pwd

'C:\\Users\\satka\\OneDrive\\Desktop\\recommendation-system\\data'

In [75]:
os.chdir("C:/Users/satka/OneDrive/Desktop/recommendation-system/data")

In [76]:
df = pd.read_csv("anime_data_24.csv")
df_links = pd.read_csv('anime_links.csv')

In [77]:
anime = df.merge(df_links,on='name')

In [78]:
anime = anime[['name','sypnopsis','image','type','episodes','status','studios','source','genres','demographic','links']]

In [79]:
#anime = anime.iloc[0:8000]

In [80]:
anime.columns

Index(['name', 'sypnopsis', 'image', 'type', 'episodes', 'status', 'studios',
       'source', 'genres', 'demographic', 'links'],
      dtype='object')

In [81]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         13501 non-null  object
 1   sypnopsis    13499 non-null  object
 2   image        13492 non-null  object
 3   type         13501 non-null  object
 4   episodes     13501 non-null  object
 5   status       13501 non-null  object
 6   studios      13501 non-null  object
 7   source       13501 non-null  object
 8   genres       11823 non-null  object
 9   demographic  13466 non-null  object
 10  links        13501 non-null  object
dtypes: object(11)
memory usage: 1.1+ MB


In [82]:
anime.dropna(inplace=True)

In [83]:
anime['sypnopsis_length'] = [len(i) for i in anime.sypnopsis]

In [84]:
anime = anime[anime['sypnopsis_length'] > 300]

In [85]:
anime['tags'] = anime['sypnopsis']+" " + anime['type']+" " + anime['episodes']+" " + anime['status'] +" "+ anime['studios'] +" "+ anime['source']+" " + anime['genres']+" " + anime['demographic']

In [117]:
anime_1 = anime.copy()
anime_1 = anime_1[['image','name','tags','links']]

In [118]:
anime_1 = anime_1.reset_index(drop=True)
anime_1 = anime_1.rename({'name':'title'},axis=1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,stop_words='english')

In [120]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [126]:
def removing_blank_lines(text):
    return text.replace('\n'," ")

def removing_pre_suff_ix(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

def converting_into_vectors(text):
    vec = cv.fit_transform(text).toarray()
    return vec

def finding_similarity(vec):
    similarity = cosine_similarity(vec)
    return similarity

In [127]:
anime_1.tags = anime_1.tags.apply(removing_blank_lines)
anime_1.tags = anime_1.tags.apply(removing_blank_lines)
vectors = converting_into_vectors(anime_1.tags)
similarity = finding_similarity(vectors)

In [131]:
anime_1[30:40]

,image,title,tags,links
30,https://cdn.myanimelist.net/images/anime/12/89...,Gintama.: Shirogane no Tamashii-hen,"after the fierc battl on rakuyou, the untold p...",https://myanimelist.net/anime/36838/Gintama__S...
31,https://cdn.myanimelist.net/images/anime/1170/...,Vinland Saga Season 2,after hi father' death and the destruct of hi ...,https://myanimelist.net/anime/49387/Vinland_Sa...
32,https://cdn.myanimelist.net/images/anime/1741/...,Monogatari Series: Off & Monster Season,koyomi araragi spent hi last year of high scho...,https://myanimelist.net/anime/57864/Monogatari...
33,https://cdn.myanimelist.net/images/anime/1792/...,Jujutsu Kaisen 2nd Season,"the year is 2006, and the hall of tokyo prefec...",https://myanimelist.net/anime/51009/Jujutsu_Ka...
34,https://cdn.myanimelist.net/images/anime/1918/...,Mob Psycho 100 II,"shigeo ""mob"" kageyama is now matur and underst...",https://myanimelist.net/anime/37510/Mob_Psycho...
35,https://cdn.myanimelist.net/images/anime/1643/...,Boku no Kokoro no Yabai Yatsu 2nd Season,"after an event winter break, kyoutar ichikawa ...",https://myanimelist.net/anime/55690/Boku_no_Ko...
36,https://cdn.myanimelist.net/images/anime/1000/...,Shingeki no Kyojin: The Final Season,gabi braun and falco grice have been train the...,https://myanimelist.net/anime/40028/Shingeki_n...
37,https://cdn.myanimelist.net/images/anime/1084/...,Kizumonogatari III: Reiketsu-hen,after help reviv the legendari vampir kiss-sho...,https://myanimelist.net/anime/31758/Kizumonoga...
38,https://cdn.myanimelist.net/images/anime/1448/...,Bocchi the Rock!,yearn to make friend and perform live with a b...,https://myanimelist.net/anime/47917/Bocchi_the...
39,https://cdn.myanimelist.net/images/anime/7/819...,Haikyuu!! Karasuno Koukou vs. Shiratorizawa Ga...,"after the victori against aoba jousai high, ka...",https://myanimelist.net/anime/32935/Haikyuu_Ka...


In [ ]:
#from sklearn.metrics.pairwise import cosine_similarity
#similarity = cosine_similarity(vectors)

In [141]:
def recommend(anime):
    anime_index = anime_1[anime_1['title']== anime].index[0]
    distances = np.around(similarity[anime_index],2)
    anime_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:8]

    for i in anime_list:
        #print(' index no. ', '|', ' title ', '|', ' similarity score ')
        print(i[0], '|', anime_1.iloc[i[0]].title, '|', i[1])
        print(anime_1.iloc[i[0]].links, '\n')
        


In [142]:
recommend('Jujutsu Kaisen 2nd Season')

3720 | Radiant | 0.28
https://myanimelist.net/anime/37202/Radiant 

6790 | Dokyuu Hentai HxEros | 0.28
https://myanimelist.net/anime/40623/Dokyuu_Hentai_HxEros 

100 | Jujutsu Kaisen | 0.27
https://myanimelist.net/anime/40748/Jujutsu_Kaisen 

169 | Jujutsu Kaisen 0 Movie | 0.25
https://myanimelist.net/anime/48561/Jujutsu_Kaisen_0_Movie 

666 | Dead Dead Demons Dededede Destruction (OVA) | 0.25
https://myanimelist.net/anime/58883/Dead_Dead_Demons_Dededede_Destruction_OVA 

1651 | Dead Dead Demons Dededede Destruction | 0.25
https://myanimelist.net/anime/51358/Dead_Dead_Demons_Dededede_Destruction 

2390 | True Tears | 0.25
https://myanimelist.net/anime/2129/True_Tears 



In [100]:
sorted(list(enumerate(similarity[anime_1[anime_1['title']== 'Shingeki no Kyojin Season 3 Part 2'].index[0]])),reverse=True,key=lambda x:x[1])[1:6]

[(720, np.float64(0.4583333333333335)),
 (123, np.float64(0.37709985557577297)),
 (42, np.float64(0.32076651393589245)),
 (26, np.float64(0.3051285766293647)),
 (110, np.float64(0.29120520167670094))]

In [101]:
anime_1.iloc[720]

image    https://cdn.myanimelist.net/images/anime/1710/...
title    Shingeki no Kyojin Season 2 Movie: Kakusei no ...
tags     eren yeager and other of the 104th train corp ...
links    https://myanimelist.net/anime/36702/Shingeki_n...
Name: 720, dtype: object